In [1]:
%pylab inline
import Stracers as st
from pygadgetreader import * 

Populating the interactive namespace from numpy and matplotlib


In [2]:
def host_sat_particles(xyz, vxyz, pids, pot, mass, Nhost_particles):
    """
    Function that return the host and the sat particles
    positions and velocities.

    Parameters:
    -----------
    xyz: snapshot coordinates with shape (n,3)
    vxys: snapshot velocities with shape (n,3)
    pids: particles ids
    Nhost_particles: Number of host particles in the snapshot
    Returns:
    --------
    xyz_mw, vxyz_mw, xyzlmc, vxyz_lmc: coordinates and velocities of
    the host and the sat.

    """
    sort_indexes = np.sort(pids)
    N_cut = sort_indexes[Nhost_particles]
    host_ids = np.where(pids<N_cut)[0]
    sat_ids = np.where(pids>=N_cut)[0]
    return xyz[host_ids], vxyz[host_ids], pot[host_ids], mass[host_ids], \
           pids[host_ids], xyz[sat_ids], vxyz[sat_ids], pot[sat_ids], \
           mass[sat_ids], pids[sat_ids]


def com_disk_potential(xyz, vxyz, Pdisk):
    V_radius = 2
    vx = vxyz[:,0]
    vy = vxyz[:,1]
    vz = vxyz[:,2]
    x = xyz[:,0]
    y = xyz[:,1]
    z = xyz[:,2]

    min_pot = np.where(Pdisk==min(Pdisk))[0]
    x_min = x[min_pot]
    y_min = y[min_pot]
    z_min = z[min_pot]
    # This >2.0 corresponds to the radius in kpc of the particles that
    # I am taking into account to compute the CM
    avg_particles = np.where(np.sqrt((x-x_min)**2.0 + (y-y_min)**2.0 + (z-z_min)**2.0)<V_radius)[0]
    x_cm = sum(x[avg_particles])/len(avg_particles)
    y_cm = sum(y[avg_particles])/len(avg_particles)
    z_cm = sum(z[avg_particles])/len(avg_particles)
    vx_cm = sum(vx[avg_particles])/len(avg_particles)
    vy_cm = sum(vy[avg_particles])/len(avg_particles)
    vz_cm = sum(vz[avg_particles])/len(avg_particles)
    return np.array([x_cm, y_cm, z_cm]), np.array([vx_cm, vy_cm, vz_cm])

def re_center(vec, cm):
    """
    Re center a halo to its center of mass.
    """
    vec[:,0] -= cm[0]
    vec[:,1] -= cm[1]
    vec[:,2] -= cm[2]
    return vec

def read_MW_snap_com_coordinates(path, snap, LMC=False, N_halo_part=0):
    MW_pos = readsnap(path+snap, 'pos', 'dm')
    MW_vel = readsnap(path+snap, 'vel', 'dm')
    dm_ids = readsnap(path+snap, 'pid', 'dm')
    MW_mass = readsnap(path+snap, 'mass', 'dm')
    MW_pot = readsnap(path+snap, 'pot', 'dm')
    print(np.max(MW_pot))

    pos_disk = readsnap(path+snap, 'pos', 'disk')
    vel_disk = readsnap(path+snap, 'vel', 'disk')
    pot_disk = readsnap(path+snap, 'pot', 'disk')

    if LMC==True:
        MW_pos, MW_vel, MW_pot, MW_mass, MW_ids, LMC_pos, LMC_vel, LMC_pot,\
        LMC_mass, LMC_ids = host_sat_particles(MW_pos, MW_vel,  dm_ids, MW_pot, \
                                              MW_mass, N_halo_part)


    pos_cm, vel_cm = com_disk_potential(pos_disk, vel_disk, pot_disk)
    MW_pos_cm = re_center(MW_pos, pos_cm)
    MW_vel_cm = re_center(MW_vel, vel_cm)
    return MW_pos_cm, MW_vel_cm, MW_pot, MW_mass, MW_ids


def truncate_halo(r, v, pos, vel, pot, mass, r_trunc):
    """
    

    """

    index_cut = np.where((r<=r_trunc) & (r>1))[0]

    return r[index_cut], v[index_cut], pos[index_cut], vel[index_cut], pot[index_cut], mass[index_cut]






In [3]:
snap = 'MWLMC6_100M_new_b0_2_000'
path = '../MW_anisotropy/code/test_snaps/'
N_halo_part = 100000000
#100000000
MW_pos, MW_vel, MW_pot, MW_mass, MW_ids = read_MW_snap_com_coordinates(path, snap, LMC=True, N_halo_part=N_halo_part)
# truncate
r_mw = np.sqrt(MW_pos[:,0]**2 + MW_pos[:,1]**2 + MW_pos[:,2]**2)
v_mw = np.sqrt(MW_vel[:,0]**2 + MW_vel[:,1]**2 + MW_vel[:,2]**2)
r_mw_tr, v_mw_tr, MW_pos_tr, MW_vel_tr, MW_pot_tr, MW_mass_tr =  truncate_halo(r_mw, v_mw, MW_pos, 
                                                                               MW_vel, MW_pot, MW_mass, 400)



Returning DM    Positions
Returning DM    Velocities
Returning DM    Particle IDs
Returning DM    Mass
Returning DM    Potential
-11112.7
Returning DISK  Positions
Returning DISK  Velocities
Returning DISK  Potential


In [15]:
rbins, pot, E, psi, Histo_E, Edges, Histo_epsilon, eps_edges = st.energies(r_mw_tr, MW_pot_tr, v_mw_tr, 1000, 100)

In [16]:
r_hr,  nu_tracer, psi_hr, dnu_dpsi_smooth, dnu2_dpsi2_smooth = st.densities_derivatives(rbins, psi, 1, interp_bins=1000, profile='Hernquist', profile_params=[40])

In [17]:
len(psi_hr)

1000

In [1]:
#def plot_derivatives():
figure(figsize=(12, 12))
subplot(2, 2, 1)
title(r'$\nu$')
loglog(r_hr, nu_tracer)
subplot(2, 2, 2)
title(r'$\Psi$')
loglog(r_hr, psi_hr)
subplot(2, 2, 3)
title(r'$d\nu/ d\Psi$')
loglog(r_hr, dnu_dpsi_smooth)
subplot(2, 2, 4)
title(r'$d^2 \nu/ d \psi^2$')
loglog(r_hr, dnu2_dpsi2_smooth)

NameError: name 'figure' is not defined